In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_excel("data\\data.xlsx")
df = df[df["LMTD"] > 0]
df = df[df["LA Chilling Duty"] >= 0]
df["SCD (MJ/kg)"] = (0.001 * df["LA Chilling Duty"]) / df["Captured CO2 (kg/h)"]

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from joblib import dump, load

In [4]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [5]:
XA = df[["FG Temperature", "Lean Loading",'Lean Amine Temperature','L/G (kg/kg)','Flue gas CO2 cons','CO2 Capture Efficiency','LMTD','Abs Height']]
XN = df[["FG Temperature", "Lean Loading",'Lean Amine Temperature','L/G (kg/kg)','Flue gas CO2 cons','CO2 Capture Efficiency']]
y = df[["SRD (MJ/kg CO2)", "Condensor Specific Duty", "SCD (MJ/kg)"]]

In [6]:
XA_train, XA_test, y_train, y_test = train_test_split(XA, y, test_size=0.30, random_state=211)
XN_train, XN_test, y_train, y_test = train_test_split(XN, y, test_size=0.30, random_state=211)

In [7]:
param_grid = {'kernel': ['rbf', 'poly'],
              'degree': [2, 3, 4],
              'gamma': ['scale','auto'],
              'C': [50, 100, 300, 500, 700, 1000, 1200, 1500]
             }

In [8]:
svr = SVR()

In [9]:
model = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=4)

In [10]:
model.fit(StandardScaler().fit_transform(XA_train), y_train['SRD (MJ/kg CO2)'])

GridSearchCV(cv=5, estimator=SVR(), n_jobs=4,
             param_grid={'C': [50, 100, 300, 500, 700, 1000, 1200, 1500],
                         'degree': [2, 3, 4], 'gamma': ['scale', 'auto'],
                         'kernel': ['rbf', 'poly']},
             scoring='neg_mean_squared_error')

In [11]:
model.best_params_

{'C': 100, 'degree': 2, 'gamma': 'auto', 'kernel': 'rbf'}

In [12]:
model.best_score_

-0.00647016248026973